# Setup

Important: Make sure you have a GPU

In [1]:
!nvidia-smi

Sun May  9 19:28:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Clone Repo and Install Dependencies


In [2]:
!git clone https://github.com/acethespy/DeeperMusicVisualizer.git
!pip install essentia
%cd /content/DeeperMusicVisualizer/

Cloning into 'DeeperMusicVisualizer'...
remote: Enumerating objects: 980, done.
remote: Counting objects: 100% (980/980), done.
remote: Compressing objects: 100% (747/747), done.
remote: Total 980 (delta 221), reused 942 (delta 190), pack-reused 0
Receiving objects: 100% (980/980), 44.75 MiB | 27.79 MiB/s, done.
Resolving deltas: 100% (221/221), done.
     |████████████████████████████████| 12.0MB 4.5MB/s 
/content/DeeperMusicVisualizer


# Rhythm

In [3]:
# Upload audio
from google.colab import files
import os, sys
import shutil

print("Upload audio file")
uploaded = files.upload()
assert len(uploaded.keys())==1, "Should only be uploading a single file"
for k in uploaded.keys():
  name = uploaded[k]
  print(type(k), k)
  print(name)
  print(type(name))
assert ".mp3" in k or ".wav" in k, "Uploaded file should be an mp3 or wav"

audio_file = "/content/DeeperMusicVisualizer/audio.mp3"
os.rename(f"{k}", "audio.mp3")

Upload audio file


Saving Faded(PaglaSongs).mp3 to Faded(PaglaSongs).mp3


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
%cd /content/DeeperMusicVisualizer/rhythm/
from act_newmap_prep import *
from act_timing import *;
from act_rhythm_calc import *
import numpy as np

model = step5_load_model("rhythm_model_8_LSTM128_overfit.h5");
params = step5_set_params(dist_multiplier=1, note_density=0.35, slider_favor=0, divisor_favor=[0] * 8, slider_max_ticks=8);

music_path = audio_file
file_path = get_timed_osu_file(music_path, game_mode=0);
step4_read_new_map(file_path);
npz = step5_load_npz();

predictions = step5_predict_notes(model, npz, params);
converted = step5_convert_sliders(predictions, params);
objs, predictions, ticks, timestamps, is_slider, is_spinner, is_note_end, sv, slider_ticks, dist_multiplier = converted;
result = []
sliding = False;
for i, pred in enumerate(predictions):
  if pred[2]==1: #slider
    slide_ticks = slider_ticks[i]
    if i == len(predictions) - 1:
      slide_ms = 1000
    elif ticks[i+1] - ticks[i] > 0:
      ms_gap = timestamps[i+1] - timestamps[i]
      slide_ms = slide_ticks / (ticks[i+1] - ticks[i]) * ms_gap
    else:
      ms_gap = timestamps[i+1] - timestamps[i]
      slide_ms = slide_ticks / (4) * ms_gap
    if sliding==False or i==len(predictions) - 1:
      result.append([timestamps[i], 1, slide_ms])
    else:
      result[-1][2] += slide_ms
    if i!=len(predictions) - 1 and ticks[i+1] - ticks[i] == slide_ticks:
      print('continuing slide')
      sliding = True;
  elif pred[1]==1: #note
    sliding=False;
    result.append([timestamps[i], 0, 0])
  else:
    #nothing for now
    continue;

rhythm_file = "/content/DeeperMusicVisualizer/rhythm.npy"
np.save(rhythm_file, result)

%cd /content/DeeperMusicVisualizer

/content/DeeperMusicVisualizer/rhythm
BPM: 90.02062225341797
Offset: 603.7188172340393
Final BPM: 90.0
(143, 16, 5)
s (2288, 5) [-0.99999976 -0.99999976 -0.99999976 -0.99999976 -0.99999976]
(2288, 11) (143, 16, 11) 8
[-0.9828572]
(2288, 1)
others (2288, 1) [0.]
another (2288, 5)
anoter [0. 0. 0. 0. 0.]
800 notes predicted.
(2288, 1) (2288, 5) (2302,) (2302,) (2288, 11)
[ 603  770  936 1103 1270]
[0 1 2 3 4]
415
(516,) (516, 5) (516,) (516,) (516,) (516,) (516,) (516,) (516,)
slider preds [1. 0. 1. 0. 0.]
slider ticks [4 4 2 2 2]
timestamps [ 603 1936 3270 4270 5936]
ticks [ 0  8 16 22 32]
1 16 3270 1.0 0.0 0.0
/content/DeeperMusicVisualizer


# Visualizer
Will need to restart runtime and re-run the notebook starting below after installing dependencies

In [1]:
%cd /content/DeeperMusicVisualizer/
# Download VisionaryArt.pkl weights for StyleGAN2
!gdown --id 14gYAkdYz78MovWOy8UIdce87-wnKC1cc
# Install dependencies
LSD_PATH = '/content/DeeperMusicVisualizer/lucid_sonic_dreams/requirements.txt'
!pip install -r $LSD_PATH

/content/DeeperMusicVisualizer
Downloading...
From: https://drive.google.com/uc?id=14gYAkdYz78MovWOy8UIdce87-wnKC1cc
To: /content/DeeperMusicVisualizer/VisionaryArt.pkl
369MB [00:01, 191MB/s]


In [3]:
from lucid_sonic_dreams.lucidsonicdreams import show_styles
from lucid_sonic_dreams.lucidsonicdreams import LucidSonicDream

style = '/content/DeeperMusicVisualizer/VisionaryArt.pkl'
audio_file = "/content/DeeperMusicVisualizer/audio.mp3"
rhythm_file = "/content/DeeperMusicVisualizer/rhythm.npy"
result_file = '/content/DeeperMusicVisualizer/results.mp4'

L = LucidSonicDream(song = audio_file,
                    pulse_audio = audio_file,
                    class_audio = audio_file,
                    contrast_audio = audio_file,
                    style = style,
                    osu_file=rhythm_file)
L.hallucinate(file_name = result_file,
              resolution = 720,
              start = 0,
              duration=15, #Can increase duration to generate a longer segment of video, but will take longer
              batch_size = 2,
              pulse_react = 0.05, 
              motion_react = 0.001)

Preparing style...
Setting up TensorFlow plugin "fused_bias_act.cu": Compiling... Loading... Done.
Preparing audio...
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.

  0%|          | 0/323 [00:00<?, ?it/s]



Hallucinating... 

Setting up TensorFlow plugin "upfirdn_2d.cu": Compiling... Loading... Done.


100%|██████████| 323/323 [03:14<00:00,  1.66it/s]


[MoviePy] >>>> Building video /content/DeeperMusicVisualizer/results.mp4
[MoviePy] Writing audio in resultsTEMP_MPY_wvf_snd.mp4


100%|██████████| 331/331 [00:00<00:00, 619.84it/s]

[MoviePy] Done.
[MoviePy] Writing video /content/DeeperMusicVisualizer/results.mp4



100%|██████████| 647/647 [00:31<00:00, 20.23it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/DeeperMusicVisualizer/results.mp4 



In [ ]:
from IPython.display import HTML
from base64 import b64encode

video_file = open(result_file, "r+b").read()

video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
HTML(f"""<video width=720 controls><source src="{video_url}"></video>""")